# **Clase sobre Automatización de Trading con IBKR API**
Este notebook te guiará paso a paso a través del código para automatizar órdenes de trading en IBKR (Interactive Brokers) usando Python. Exploraremos cómo conectarnos a la API, manejar datos en tiempo real, crear órdenes de mercado, gestionar las órdenes activas, y más.

## **1. Clase IBKRConnection**
La clase `IBKRConnection` extiende `EClient` y `EWrapper` para establecer la conexión con la API de IBKR y manejar la comunicación.

### Atributos:
- **`data_handlers`**: Un diccionario que mapea tickers a instancias de `DataHandler`, las cuales gestionan la información del mercado.
- **`order_id_counter`**: Contador para generar identificadores únicos para las órdenes.
- **`active_orders`**: Almacena el estado de las órdenes activas por ticker.
- **`executed_price`**: Precio de la última ejecución de una orden.
- **`current_contract`**: Contrato actual para el que se envían las órdenes.
- **`pending_orders`**: Almacena información sobre órdenes pendientes.
- **`order_map`**: Mapeo de ticker a `orderId`.
- **`brackets_pendientes`**: Estado para rastrear órdenes de brackets pendientes.
- **`parentIds`**: Lista que almacena los IDs de las órdenes padre para rastreo.

### Métodos Principales:
- **`tickPrice`**: Maneja actualizaciones de precios en tiempo real.
- **`tickSize`**: Maneja actualizaciones del tamaño del tick.
- **`cancelar_ordenes_pendientes`**: Cancela todas las órdenes activas al inicio.
- **`cancelar_orden`**: Cancela una orden activa específica.
- **`enviar_orden`**: Envía una nueva orden de trading.
- **`nextValidId`**: Actualiza el siguiente ID válido para las órdenes.

In [1]:
class IBKRConnection(EWrapper, EClient):
    def __init__(self, data_handlers):
        EClient.__init__(self, self)

NameError: name 'EWrapper' is not defined

In [ ]:
    def tickPrice(self, reqId: TickerId, tickType: TickType, price: float, attrib: TickAttrib):
        if tickType == 4:  # Si es el último precio
            if reqId in self.data_handlers:
                self.data_handlers[reqId].add_tick(price)  # Añade el precio al manejador de datos
                self.data_handlers[reqId].last_tick = price  # Guarda el último precio

    def tickSize(self, reqId: TickerId, tickType: TickType, size: Decimal):
        if tickType == 8:  # Si es el volumen
            if reqId in self.data_handlers:
                if self.counter_vol == 0:
                    self.vol_ref = size  # Guarda la referencia de volumen
                else:
                    self.data_handlers[reqId].add_volume(size - self.vol_ref)  # Añade la diferencia de volumen
                self.vol_ref = size  # Actualiza la referencia
                self.counter_vol += 1  # Incrementa el contador de volumen
                print('tick')  # Imprime cuando recibe un tick


In [ ]:
    def cancelar_ordenes_pendientes(self):
        print("Cancelando todas las órdenes activas al iniciar.")
        oc = OrderCancel()  # Crea un objeto de cancelación de órdenes
        self.reqGlobalCancel(oc)  # Cancela todas las órdenes activas en la cuenta
        self.brackets_pendientes = {}  # Restablece el estado de brackets pendientes

    def enviar_orden(self, stock, cantidad, direccion, dif):
        ticker = stock['ticker']
        oc = OrderCancel()  # Crea un objeto de cancelación

        # Verifica si ya hay órdenes pendientes
        if ticker in self.brackets_pendientes and self.brackets_pendientes[ticker]:
            print(f"No se puede enviar una nueva orden para {ticker}: hay órdenes de bracket pendientes.")
            return

        reqId = [key for key, val in self.data_handlers.items() if val.ticker['ticker'] == stock['ticker']][0]
        ultimo_tick = self.data_handlers[reqId].last_tick  # Último precio del ticker

        contrato = crear_contrato(stock)
        self.current_contract = contrato
        order_id = self.nextOrderId()
        ordenes = create_orden_market_con_bracket(order_id, direccion, cantidad, dif, ultimo_tick)  # Crea la orden

        # Envía las órdenes
        for orden in ordenes:
            self.placeOrder(orden.orderId, contrato, orden)
            self.nextOrderId()  # Obtiene el siguiente OrderId
            print(f"Orden enviada: {direccion} {cantidad} acciones de {ticker}")


In [ ]:
    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"Estado de la orden ID {orderId}: {status}, llenada: {filled}, restante: {remaining}")

        if status in ['Cancelled', 'Inactive']:  # Si la orden fue cancelada
            return

        if status == 'Filled' and remaining <= 0.00000001:  # Si la orden fue completamente llenada
            if orderId not in self.parentIds:
                if parentId == 0:
                    self.parentIds.append(orderId)
                    self.brackets_pendientes[self.current_contract.symbol] = True
                else:
                    self.brackets_pendientes[self.current_contract.symbol] = False  # Marca el bracket como completado


In [ ]:
    def nextValidId(self, orderId: int):
        self.order_id_counter = orderId  # Actualiza el contador de IDs
        print(f"Siguiente OrderId válido: {orderId}")

    def nextOrderId(self):
        current_id = self.order_id_counter  # Obtiene el ID actual
        self.order_id_counter += 1  # Incrementa el ID para la siguiente orden
        return current_id


## **2. Clase DataHandler**
La clase `DataHandler` se encarga de gestionar los datos de precios y volúmenes para cada ticker.

### Atributos:
- **`ticker`**: Información sobre el ticker.
- **`ticks`**: Lista de precios en tiempo real recibidos.
- **`volumen_acumulado`**: Volumen total acumulado desde la última barra.
- **`barras`**: Lista de barras OHLC (Open, High, Low, Close) generadas.
- **`threshold_bar`**: Umbral de volumen para generar nuevas barras.
- **`threshold_features`**: Umbral para calcular características y hacer predicciones.
- **`connection`**: Referencia a la conexión IBKR.
- **`model`**: Modelo de predicción cargado.
- **`scaler`**: Escalador para normalizar los datos de entrada.
- **`last_tick`**: Último precio recibido.

### Métodos Principales:
- **`add_tick`**: Agrega un nuevo precio a la lista de ticks y verifica si se puede generar una nueva barra.
- **`add_volume`**: Agrega volumen y verifica si se puede generar una nueva barra.
- **`check_if_bar_can_be_generated`**: Comprueba si se puede generar una nueva barra OHLC.
- **`generar_barra_ohlc`**: Crea una nueva barra OHLC y la añade a la lista.
- **`crear_features_y_predecir`**: Calcula características basadas en las barras y realiza predicciones.
- **`evaluar_y_enviar_orden`**: Evalúa la predicción y envía una orden si es necesario.

In [ ]:
    def add_tick(self, price):
        self.ticks.append(price) 
        self.check_if_bar_can_be_generated()  
  
    def add_volume(self, volume):
        self.volumen_acumulado += volume 
        self.check_if_bar_can_be_generated()  


In [ ]:
   def check_if_bar_can_be_generated(self):
        if self.volumen_acumulado >= self.threshold_bar:  
            self.generar_barra_ohlc() 

    
    def generar_barra_ohlc(self):
        if self.ticks:  # Solo genera la barra si hay precios en la lista de ticks
            # Calcula los precios OHLC (Open, High, Low, Close)
            open_price = self.ticks[0]  # Precio de apertura (primer tick)
            high_price = max(self.ticks)  # Precio más alto
            low_price = min(self.ticks)  # Precio más bajo
            close_price = self.ticks[-1]  # Precio de cierre (último tick)

            # Crea una barra OHLC con el volumen acumulado
            barra = {
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price,
                'volume': self.volumen_acumulado
            }
            self.barras.append(barra)  
            self.ticks.clear()  
            self.volumen_acumulado = 0

            print(f"Barra generada para {self.ticker['ticker']}: {barra}")

            # Si el número de barras generadas supera el umbral para calcular features, inicia ese proceso
            if len(self.barras) > self.threshold_features:
                self.crear_features_y_predecir()


In [ ]:
    # Método para calcular los features basados en las barras OHLC y hacer predicciones con el modelo
    def crear_features_y_predecir(self):
        # Selecciona las últimas "threshold_features" barras y crea un DataFrame
        df_barras = pd.DataFrame(self.barras[-self.threshold_features:])
        
        # Calcula los features a partir de las barras
        features = calcular_features(df_barras, self.threshold_features)  
        df_barras = pd.DataFrame(features).iloc[:,-1]  

        # Escala los features usando el escalador antes de pasarlos al modelo de predicción
        features_scaled = self.scaler.transform([df_barras])  

        # Usa el modelo de predicción para obtener una predicción basada en los features escalados
        prediccion = self.model.predict(features_scaled)

        print(f"Features creados para {self.ticker['ticker']}: {features_scaled}")
        print("PREDICCION:", prediccion)

        # Variables de ejemplo para cantidad de la orden y diferencial de precio
        cantidad = 1000  
        diferencial = 10 

        # Evalúa la predicción y decide si enviar una orden de compra o venta
        self.evaluar_y_enviar_orden(prediccion, cantidad, diferencial)

    # Método para evaluar la predicción y enviar una orden basada en la misma
    def evaluar_y_enviar_orden(self, prediccion, cantidad, diferencial):
        # Si la predicción es 1, envía una orden de compra
        if prediccion == 1:
            print(f"Enviando orden de compra para {self.ticker['ticker']}")
            self.connection.enviar_mensaje(self.ticker, cantidad, 'BUY', diferencial)  
        # Si la predicción es -1, envía una orden de venta
        elif prediccion == -1:
            print(f"Enviando orden de venta para {self.ticker['ticker']}")
            self.connection.enviar_mensaje(self.ticker, cantidad, 'SELL', diferencial) 

## **3. Creación de Órdenes**
Se utilizan funciones para crear diferentes tipos de órdenes, incluyendo órdenes de mercado y órdenes de bracket.

### Funciones Principales:
- **`create_orden_market_con_bracket`**: Crea una orden de mercado con un bracket de ganancias y pérdidas.

### Estructura de Órdenes:
- **Órdenes de Mercado**: Se ejecutan al mejor precio disponible en el mercado.
- **Órdenes de Bracket**: Combinan una orden de mercado con órdenes limitadas de toma de ganancias y stop loss.


## **4. Conexión y Gestión de Datos**
El flujo principal del programa incluye la conexión a IBKR y la gestión de datos en tiempo real para diferentes tickers.

### Pasos:
1. **Conexión a la API de IBKR**: Establece la conexión utilizando el cliente de IBKR.
2. **Registro de Manejadores de Datos**: Crea instancias de `DataHandler` para cada ticker.
3. **Solicitar Datos de Mercado**: Envía solicitudes para obtener datos en tiempo real de los tickers registrados.
4. **Gestión de Órdenes**: Envía, cancela y gestiona el estado de las órdenes activas.


## **5. Ejecución del Código**
El código se ejecuta en un hilo separado para mantener la conexión activa y procesar datos en tiempo real.

### Componentes Clave:
- **Hilo de API**: Se utiliza un hilo separado para ejecutar el bucle de la API y manejar eventos de la API sin bloquear el hilo principal.
- **Manejo de Excepciones**: Se implementan excepciones para manejar desconexiones y errores en la conexión.


In [ ]:
if __name__ == "__main__":
    # Definir los manejadores de datos para cada ticker
    data_handlers = {i: DataHandler(symbol, threshold_bar, threshold_features, None, models[0], scalers[0]) for i, symbol in symbols.items()}    
    
    # Iniciar la conexión a IBKR
    app = IBKRConnection(data_handlers)
    app.connect("127.0.0.1", 7497, 0)
    time.sleep(1)

    # Iniciar el loop de la API en un hilo separado
    api_thread = threading.Thread(target=app.run, daemon=True)
    api_thread.start()

    # Solicitar datos de mercado para cada ticker
    for reqId, ticker in enumerate(list(symbols.values())):
        contrato = crear_contrato(ticker)
        app.reqMktData(reqId, contrato, "", False,


## **6. Otros**

### Componentes Clave:
- **Creación de contrato**: Es el "activo" básico con el que se solicita y recibe información
- **Creación de features**: Para fines prácticos usaremos features técnicos

In [ ]:
def create_orden_market(orderid, direction, quantity:

    orden_market = Order()  # Instancia una nueva orden
    orden_market.orderId = orderid 
    orden_market.action = direction
    orden_market.orderType = "MKT"
    orden_market.totalQuantity = quantity
    orden_market.tif = "IOC"  
    orden_market.transmit = True

    return orden_market